<a href="https://colab.research.google.com/github/VivekSivaramakrishnan/Depression-AI/blob/master/(Text)Feature_Extraction_and_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
import nltk.data
import string
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
word_tokenizer = TweetTokenizer()
lemmatizer = WordNetLemmatizer()

# Example
lemmatizer.lemmatize('geese')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'goose'

In [0]:
stop_words = set(stopwords.words('english'))

In [0]:
!wget https://github.com/singhalprerana/SST_data_extraction/archive/master.zip
  
import zipfile
zip_ref = zipfile.ZipFile('master.zip', 'r')
zip_ref.extractall()
zip_ref.close()

--2019-03-17 06:36:40--  https://github.com/singhalprerana/SST_data_extraction/archive/master.zip
Resolving github.com (github.com)... 192.30.253.112, 192.30.253.113
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/singhalprerana/SST_data_extraction/zip/master [following]
--2019-03-17 06:36:40--  https://codeload.github.com/singhalprerana/SST_data_extraction/zip/master
Resolving codeload.github.com (codeload.github.com)... 192.30.253.121, 192.30.253.120
Connecting to codeload.github.com (codeload.github.com)|192.30.253.121|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip.1’

master.zip.1            [       <=>          ]  13.31M  5.18MB/s    in 2.6s    

2019-03-17 06:36:43 (5.18 MB/s) - ‘master.zip.1’ saved [13955306]



In [0]:
!ls SST_data_extraction-master/

datasetSentences.txt	  SOStr.txt		    sst_test.csv
datasetSplit.txt	  sst5_dev.csv		    sst_train_phrases.csv
dictionary.txt		  sst5_test.csv		    sst_train_sentences.csv
original_rt_snippets.txt  sst5_train_phrases.csv    STree.txt
README.txt		  sst5_train_sentences.csv  xtract_sst.py
sentiment_labels.txt	  sst_dev.csv


In [0]:
# This file includes:
# 1. original_rt_snippets.txt contains 10,605 processed snippets from the original pool of Rotten Tomatoes HTML files. Please note that some snippet may contain multiple sentences.

# 2. dictionary.txt contains all phrases and their IDs, separated by a vertical line |

# 3. sentiment_labels.txt contains all phrase ids and the corresponding sentiment labels, separated by a vertical line.
# Note that you can recover the 5 classes by mapping the positivity probability using the following cut-offs:
# [0, 0.2], (0.2, 0.4], (0.4, 0.6], (0.6, 0.8], (0.8, 1.0]
# for very negative, negative, neutral, positive, very positive, respectively.
# Please note that phrase ids and sentence ids are not the same.

# 4. SOStr.txt and STree.txt encode the structure of the parse trees. 
# STree encodes the trees in a parent pointer format. Each line corresponds to each sentence in the datasetSentences.txt file. The Matlab code of this paper will show you how to read this format if you are not familiar with it.

# 5. datasetSentences.txt contains the sentence index, followed by the sentence string separated by a tab. These are the sentences of the train/dev/test sets.

# 6. datasetSplit.txt contains the sentence index (corresponding to the index in datasetSentences.txt file) followed by the set label separated by a comma:
# 	1 = train
# 	2 = test
# 	3 = dev

# Please note that the datasetSentences.txt file has more sentences/lines than the original_rt_snippet.txt. 
# Each row in the latter represents a snippet as shown on RT, whereas the former is each sub sentence as determined by the Stanford parser.

In [0]:
import pandas as pd
import string

lst = pd.read_csv('SST_data_extraction-master/sst_train_sentences.csv')
lst.columns = ['labels', 'features']
lst.head(10)

,labels,features
0,0.70833,"Despite the film 's shortcomings , the stories..."
1,0.26389,"Despite its dry wit and compassion , the film ..."
2,0.47222,The central character is n't complex enough to...
3,0.13889,Rifkin no doubt fancies himself something of a...
4,0.45833,"Dodgy mixture of cutesy romance , dark satire ..."
5,0.51389,The only time 8 Crazy Nights comes close to hi...
6,0.70833,... quite good at providing some good old fash...
7,0.73611,"The huskies are beautiful , the border collie ..."
8,0.41667,There is so much plodding sensitivity .
9,0.66667,"At the end , when the now computerized Yoda fi..."


In [0]:
lst[:1].features[0]

"Despite the film 's shortcomings , the stories are quietly moving ."

In [0]:
lst['features'] = lst['features'].apply(refine_sentence)

In [0]:
lst[:1].features[0]

['story', 'moving', 'Despite', 'quietly', 'shortcoming', 'film']

In [0]:
def refine_sentence(word):
    table = str.maketrans('', '', string.punctuation)
    tokenized = word_tokenizer.tokenize(word)
    cleaned_and_stemmed = [lemmatizer.lemmatize(i) for i in tokenized if i not in stop_words]
    no_punctuation = list(set([i.translate(table) for i in cleaned_and_stemmed if i.translate(table)]))
    return no_punctuation

In [0]:
refined_lst = lst['features'].tolist()

In [0]:
freqDist = nltk.FreqDist([i for j in refined_lst for i in j])

In [0]:
bag_of_words = sorted(list(freqDist), key=lambda i: freqDist[i])[::-1]
# Top 100 words for feature extraction
top_words = bag_of_words[:5000]
top_words

['film',
 'The',
 'movie',
 'A',
 'It',
 'nt',
 'one',
 'like',
 'story',
 'make',
 'I',
 'character',
 'comedy',
 'time',
 'This',
 'much',
 'good',
 'work',
 'performance',
 'way',
 'even',
 'funny',
 'little',
 'An',
 'life',
 'would',
 'never',
 'feel',
 'director',
 'get',
 'may',
 'enough',
 'u',
 'look',
 'come',
 'thing',
 'best',
 'If',
 'could',
 'There',
 'drama',
 'love',
 'made',
 'year',
 'plot',
 'bad',
 'audience',
 'take',
 'really',
 'something',
 'many',
 'well',
 'action',
 'minute',
 'better',
 'see',
 'give',
 'without',
 'go',
 'What',
 'people',
 'end',
 'also',
 'still',
 'cast',
 'might',
 'picture',
 'moment',
 'actor',
 'script',
 'two',
 'fun',
 'every',
 'sense',
 'long',
 'great',
 'nothing',
 'humor',
 'new',
 'ever',
 'first',
 'thriller',
 'scene',
 'lot',
 'In',
 'As',
 'documentary',
 'kind',
 'But',
 'find',
 'You',
 'tale',
 'often',
 'real',
 'entertaining',
 'seems',
 'One',
 'another',
 'heart',
 'quite',
 'Hollywood',
 'le',
 'show',
 'almost',

In [0]:
import pickle
with open('topWords.pkl', 'wb') as f:
  pickle.dump(top_words, f)

In [0]:
features = []
for sentence in refined_lst:
    features.append([int(top_word in sentence) for top_word in top_words])

In [0]:
labels = [int(i>=0.5) for i in lst['labels'].tolist()]

In [0]:
# Shuffle the features and labels
import random
lsst = list(zip(features, labels))
random.shuffle(lsst)
features, labels = zip(*lsst)

In [0]:
# int(label_frame['sentiment values'].tolist()[13] >= 0.6)
len(features), len(labels)

(8533, 8533)

In [0]:
# Training
X, y = list(features), list(labels)

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4, random_state=42)

names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

# Testing
for name, clf in zip(names, classifiers):
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    print('{} scored: {}'.format(name, score))


Nearest Neighbors scored: 0.5351493848857645
Linear SVM scored: 0.6549502050380785
RBF SVM scored: 0.526654950205038
Gaussian Process scored: 0.5222612770943175
Decision Tree scored: 0.5556531927357938
Random Forest scored: 0.5199179847685999
Neural Net scored: 0.7126537785588752
AdaBoost scored: 0.6004686584651435
Naive Bayes scored: 0.6063268892794376


/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


QDA scored: 0.5536028119507909


In [0]:
# Nearest Neighbors scored: 0.5351493848857645
# Linear SVM scored: 0.6549502050380785
# RBF SVM scored: 0.526654950205038
# Gaussian Process scored: 0.5222612770943175
# Decision Tree scored: 0.5556531927357938
# Random Forest scored: 0.5199179847685999
# Neural Net scored: 0.7126537785588752
# AdaBoost scored: 0.6004686584651435
# Naive Bayes scored: 0.6063268892794376
# QDA scored: 0.5536028119507909


In [0]:
# Testing with user input
while True:
    t = input('Enter sentence: ')
    if not t: break
    name, clf = names[6], classifiers[6]
        
    ready_t = [int(top_word in refine_sentence(t)) for top_word in top_words]

    ans = ['Depressed', 'Happy'][clf.predict([ready_t])[0]]

    print('{} thinks you are: {}\n'.format(name, ans))

Enter sentence: Head over heels
Neural Net thinks you are: Happy

Enter sentence: Best day ever
Neural Net thinks you are: Happy

Enter sentence: Happy as a clam
Neural Net thinks you are: Happy

Enter sentence: I am so excited today
Neural Net thinks you are: Depressed

Enter sentence: I can't wait to go out
Neural Net thinks you are: Depressed

Enter sentence: I am so happy
Neural Net thinks you are: Happy

Enter sentence: Lonely, that's all I am at this point
Neural Net thinks you are: Depressed

Enter sentence: I'm slowly giving up.
Neural Net thinks you are: Happy

Enter sentence: I have cried becaure you are you
Neural Net thinks you are: Depressed

Enter sentence: 


In [0]:
# Save model for use
from sklearn.externals import joblib
for name, clf in zip(names, classifiers):
  joblib.dump(clf, name+'.pkl')